In [1]:
import get_reviews_with_scrapy
import pandas as pd
from scrapy.crawler import CrawlerProcess
import numpy as np

In [2]:
class SteamspiderSpider(get_reviews_with_scrapy.Spider):
    name = "steamspider"
    allowed_domains = ["store.steampowered.com"]

    def start_requests(self):
        self.df = pd.read_csv("data/steam_data_applied_set_date_func.csv")
        self.df["review_text"] = None
        self.df["review_value"] = None
        df_temp = self.df.head(100)  # İlk 5000 satırı alıyoruz, isteğe bağlı olarak değiştirebilirsiniz
        url_list = df_temp["Link"].str.strip()

        for url in url_list:
            yield get_reviews_with_scrapy.Request(url=url, callback=self.parse, meta={'url': url})

    def parse(self, response):
        name = response.css('div#appHubAppName.apphub_AppName::text').get()
        review_summary = response.css('div.summary_section span::text').getall()
        url = response.meta['url']

        if name and review_summary:
            review_text = review_summary[0].strip()
            review_value = review_summary[1].strip()
            self.df.loc[self.df['Link'] == url, 'review_text'] = review_text
            self.df.loc[self.df['Link'] == url, 'review_value'] = review_value
        
        else:
            # Hata durumunda da DataFrame'e ekleyin
            self.df.loc[self.df['Link'] == url, 'review_text'] = np.nan
            self.df.loc[self.df['Link'] == url, 'review_value'] = np.nan
        self.df.to_csv("data/steam_data_applied_set_date_func_updated.csv", index=False)

    def closed(self, reason):
        print("All processing is done.")

In [3]:
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'output.json'
})
process.crawl(SteamspiderSpider)
process.start()

2024-05-14 00:24:19 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-05-14 00:24:19 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1m  14 Dec 2021), cryptography 36.0.1, Platform Windows-10-10.0.19045-SP0
2024-05-14 00:24:19 [scrapy.addons] INFO: Enabled addons:
[]
2024-05-14 00:24:19 [py.warnings] WARNING: c:\Users\Sadık\AppData\Local\Programs\Python\Python39\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy

All processing is done.


In [8]:
# file_path = "/content/drive/MyDrive/Applied Machine Learning/data/steam_data_applied_set_date_func.csv"
# df = pd.read_csv(file_path)
# df_temp = df.head(5000)
# url_list = df_temp["Link"].str.strip()

# for url in url_list:
#     yield scrapy.Request(url=url, callback=self.parse, meta={'url': url})

In [7]:
url_list = df_temp["Link"].tolist()

In [9]:
df

,Title,Original Price,Discounted Price,Release Date,Link,Game Description,Recent Reviews Summary,All Reviews Summary,Recent Reviews Number,All Reviews Number,Developer,Publisher,Supported Languages,Popular Tags,Game Features,Minimum Requirements
0,Baldur's Gate 3,$29.99,$29.99,2023-08-03,https://store.steampowered.com/app/1086940/Bal...,"Baldur’s Gate 3 is a story-rich, party-based R...",Overwhelmingly Positive,Very Positive,"- 96% of the 128,900 user reviews in the last ...","- 94% of the 188,617 user reviews for this gam...",Larian Studios,Larian Studios,"['English', 'French', 'German', 'Spanish - Spa...","['RPG', 'Choices Matter', 'Character Customiza...","['Single-player', 'Online Co-op', 'LAN Co-op',...",Requires a 64-bit processor and operating syst...
1,Counter-Strike: Global Offensive,$14.99,$14.99,2012-08-21,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive (CS: GO) expa...,Very Positive,Very Positive,"- 89% of the 75,284 user reviews in the last 3...","- 88% of the 7,428,921 user reviews for this g...","Valve, Hidden Path Entertainment",Valve,"['English', 'Czech', 'Danish', 'Dutch', 'Finni...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['Steam Achievements', 'Full controller suppor...",OS: | Windows® 7/Vista/XP | Processor: | Int...
2,Apex Legends™,Free,Free,2020-11-04,https://store.steampowered.com/app/1172470/Ape...,"Apex Legends is the award-winning, free-to-pla...",Mixed,Very Positive,"- 65% of the 18,581 user reviews in the last 3...","- 80% of the 701,597 user reviews for this gam...",Respawn Entertainment,Electronic Arts,"['English', 'French', 'Italian', 'German', 'Sp...","['Free to Play', 'Multiplayer', 'Battle Royale...","['Online PvP', 'Online Co-op', 'Steam Achievem...",Requires a 64-bit processor and operating syst...
3,Forza Horizon 5,$34.78,$17.39,2021-11-08,https://store.steampowered.com/app/1551360/For...,Your Ultimate Horizon Adventure awaits! Explor...,Very Positive,Very Positive,"- 87% of the 4,120 user reviews in the last 30...","- 88% of the 123,162 user reviews for this gam...",Playground Games,Xbox Game Studios,"['English', 'French', 'Italian', 'German', 'Sp...","['Racing', 'Open World', 'Driving', 'Multiplay...","['Single-player', 'Online PvP', 'Online Co-op'...",Requires a 64-bit processor and operating syst...
4,Call of Duty®,Free,Free,2022-10-27,https://store.steampowered.com/app/1938090/Cal...,"Welcome to Call of Duty® HQ, the home of Call ...",Mixed,Mixed,"- 49% of the 8,257 user reviews in the last 30...","- 57% of the 236,876 user reviews for this gam...","Infinity Ward, Raven Software, Beenox, Treyarc...",Activision,"['English', 'French', 'Italian', 'German', 'Sp...","['FPS', 'Multiplayer', 'Shooter', 'Action', 'S...","['Single-player', 'Online PvP', 'Online Co-op'...",Requires a 64-bit processor and operating syst...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61884,Cruo Domine,Free,Free,2023-04-21,https://store.steampowered.com/app/2360710/Cru...,Survive in a hostile space environment with ra...,3 user reviews,NaN,- Need more user reviews to generate a score,NaN,Jiri Otoupal,Jiri Otoupal,['English'],"['Early Access', 'Action', 'Adventure', 'RPG',...","['Single-player', 'Steam Achievements', 'Track...",Requires a 64-bit processor and operating syst...
61885,Retchid,$12.49,$12.49,2023-08-18,https://store.steampowered.com/app/1419060/Ret...,You are a TEC Engineer from Earth ordered to r...,2 user reviews,NaN,- Need more user reviews to generate a score,NaN,From Beneath Software,From Beneath Software,['English'],"['Early Access', 'Action', 'FPS', 'Shooter', '...","['Single-player', 'Partial Controller Support']",Requires a 64-bit processor and operating syst...
61886,Dungeon Legend,$14.49,$14.49,2023-04-01,https://store.steampowered.com/app/2360670/Dun...,This is a casual breakout game. The pace of th...,NaN,NaN,NaN,NaN,Louis Winfield Game Studios,Louis Winfield Game Studios,['English'],"['Casual', 'Rhythm', '3D Platforme